# Problema de designação de pessoas **I** para instrumentos **J** e vozes **L** no dia **K**
---

##  i - Conjunto de pessoas
##  J - Conjunto de instrumentos
##  k - Conjunto de dias
##  l - Conjunto de vozes

---

* Mij = Matriz Binária - Pessoa I toca Instrumento J

* Sil = Matriz Binária - Pessoa I toca canta na posição L

* PMj = Peso de cada instrumento J
* PSl = Peso de cada voz L

* Dk = Peso de cada dia K

### Solução
* Xijk - Matriz Binária no R3
* Yilk - Matriz Binária no R3
* Zik -  Matriz binária no R2

* **Se Xijk = 1 => A pessoa "i" tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 1 => A pessoa "i" cantará na posição 'l' no dia 'k'

* **Se Xijk = 0 => A pessoa "i" não tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 0 => A pessoa "i" não cantará na posição 'l' no dia 'k'

* **Somatório em i de Xijk <= 1 -  Um instrumento 'j' só pode ser tocado por uma pessoa 'i' em um dia 'k'**

* Somatório em i de Yilk <= 1 - Uma voz 'l' só pode ser cantada por uma pessoa 'i' em um dia 'k'

* **Somatório em 'j' de Xijk <= 1 - A pessoa "i" tocará somente um instrumento instrumento 'j' no dia 'k'**

* Somatório em 'l' de Yilk <= 1 - A pessoa "i" cantará somente em uma posição "l" no dia "k"

* **Zik - Pessoa i participa no dia k**
* **Zik = 0 se Xijk e yilk = 0**
* **Zik = 1 se Xijk ou yilk = 1**


* **Zik + Zi(k+1) + Zi(k+2) + Zi(k+4) <= 2 para todo i - A pessoa "i" não participará mais do que dois dias seguidos.**


* **Xijk = 0 se PMij = 0**
* Yilk = 0 se PSil = 0


* Maximize Xijk * Mij * PMj * Dk + Yilk * Sil * PSl * Dk

In [1]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Peso dos Dias','Disponibilidade', 'Dias por Mês', 'Vozes'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
D = pd.Series(sheets_dict['Peso dos Dias']['Peso'])

Disp = sheets_dict['Disponibilidade']
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

In [2]:
from pulp import *

days = 2

# Definir o problema
prob = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (P.index, P.columns, D.index), cat='Binary')

# Função objetivo
prob += lpSum([X[i][j][k] * P.loc[i, j] * L[j] * D[k] for i in P.index for j in P.columns for k in D.index])

# Restrições
for j in P.columns:
    for k in D.index:
        prob += lpSum([X[i][j][k] for i in P.index]) <= 1

for i in P.index:
    for k in D.index:
        prob += lpSum([X[i][j][k] for j in P.columns]) <= 1

for i in P.index:
    for k in range(len(D.index) - 3):
        prob += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]] for j in P.columns]) <= DPM['Quantidade'][i]

for i in P.index:
    for j in P.columns:
        for k in D.index:
            if P.loc[i, j] == 0:
                prob += X[i][j][k] == 0

# Resolver o problema
prob.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hmbarros/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/21117631e0cd4dc999acc74fb9e9447f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/21117631e0cd4dc999acc74fb9e9447f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1355 COLUMNS
At line 9906 RHS
At line 11257 BOUNDS
At line 12428 ENDATA
Problem MODEL has 1350 rows, 1170 columns and 5911 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1326 - 0.00 seconds
Cgl0002I 871 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 92 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 66 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 39 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 33 strengthened rows, 0 substitutions

1

In [3]:
# Imprimir o status da solução
print("Status:", LpStatus[prob.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob.objective))

# Dicionário para armazenar os resultados
resultados = {}

# Iterar sobre cada variável de decisão
for v in prob.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados:
            resultados[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados[dia][instrumento] = pessoa

print(resultados)
df = pd.DataFrame(columns=P.columns, index=sheets_dict['Peso dos Dias']['Dias'])

# Preencher o DataFrame com os resultados
for dia in resultados:
    for instrumento in resultados[dia]:
        df.loc[sheets_dict['Peso dos Dias']['Dias'][int(dia)], instrumento] = resultados[dia][instrumento]

# Substituir NaN por ''
df = df.fillna('')

if not pd.api.types.is_datetime64_any_dtype(df.index):
    df.index = pd.to_datetime(df.index, origin='1899-12-30', unit='D')

# Formatar o índice para o formato brasileiro
df.index = df.index.strftime('%d/%m/%Y')

df

Status: Optimal
Valor da função objetivo = 1326.0
{'11': {'Teclado': 'Bernardo', 'Bateria': 'Lucas', 'Guitarra': 'Osvaldo', 'Baixo': 'Rafael', 'Violão': 'Ricardo'}, '2': {'Teclado': 'Bernardo', 'Baixo': 'Gabriel', 'Bateria': 'Lucas', 'Guitarra': 'Osvaldo', 'Violão': 'Ricardo'}, '5': {'Teclado': 'Bernardo', 'Guitarra': 'Enzo', 'Bateria': 'Lucas', 'Violão': 'Osvaldo', 'Baixo': 'Ricardo'}, '8': {'Teclado': 'Bernardo', 'Guitarra': 'Daniel', 'Bateria': 'Lucas', 'Violão': 'Osvaldo', 'Baixo': 'Ricardo'}, '1': {'Baixo': 'Daniel', 'Guitarra': 'Enzo', 'Bateria': 'Marcela', 'Violão': 'Osvaldo', 'Teclado': 'Vinicius'}, '0': {'Guitarra': 'Daniel', 'Baixo': 'Enzo', 'Teclado': 'Henrique', 'Bateria': 'Israel', 'Violão': 'Ricardo'}, '4': {'Guitarra': 'Daniel', 'Violão': 'Henrique', 'Bateria': 'Marcela', 'Baixo': 'Osvaldo', 'Teclado': 'Vinicius'}, '9': {'Guitarra': 'Enzo', 'Baixo': 'Gabriel', 'Bateria': 'Henrique', 'Violão': 'Osvaldo', 'Teclado': 'Vinicius'}, '6': {'Baixo': 'Gabriel', 'Bateria': 'Henriq

,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
14/01/2024,Israel,Enzo,Henrique,Ricardo,Daniel
21/01/2024,Marcela,Daniel,Vinicius,Osvaldo,Enzo
28/01/2024,Lucas,Gabriel,Bernardo,Ricardo,Osvaldo
04/02/2024,Henrique,Ricardo,Vinicius,Rafael,Osvaldo
11/02/2024,Marcela,Osvaldo,Vinicius,Henrique,Daniel
18/02/2024,Lucas,Ricardo,Bernardo,Osvaldo,Enzo
25/02/2024,Henrique,Gabriel,Vinicius,Rafael,Osvaldo
03/03/2024,Marcela,Ricardo,Vinicius,Henrique,Osvaldo
10/03/2024,Lucas,Ricardo,Bernardo,Osvaldo,Daniel


In [145]:
prob.variables()

[X_Ariane_Baixo_0,
 X_Ariane_Baixo_1,
 X_Ariane_Baixo_10,
 X_Ariane_Baixo_11,
 X_Ariane_Baixo_12,
 X_Ariane_Baixo_2,
 X_Ariane_Baixo_3,
 X_Ariane_Baixo_4,
 X_Ariane_Baixo_5,
 X_Ariane_Baixo_6,
 X_Ariane_Baixo_7,
 X_Ariane_Baixo_8,
 X_Ariane_Baixo_9,
 X_Ariane_Bateria_0,
 X_Ariane_Bateria_1,
 X_Ariane_Bateria_10,
 X_Ariane_Bateria_11,
 X_Ariane_Bateria_12,
 X_Ariane_Bateria_2,
 X_Ariane_Bateria_3,
 X_Ariane_Bateria_4,
 X_Ariane_Bateria_5,
 X_Ariane_Bateria_6,
 X_Ariane_Bateria_7,
 X_Ariane_Bateria_8,
 X_Ariane_Bateria_9,
 X_Ariane_Guitarra_0,
 X_Ariane_Guitarra_1,
 X_Ariane_Guitarra_10,
 X_Ariane_Guitarra_11,
 X_Ariane_Guitarra_12,
 X_Ariane_Guitarra_2,
 X_Ariane_Guitarra_3,
 X_Ariane_Guitarra_4,
 X_Ariane_Guitarra_5,
 X_Ariane_Guitarra_6,
 X_Ariane_Guitarra_7,
 X_Ariane_Guitarra_8,
 X_Ariane_Guitarra_9,
 X_Ariane_Teclado_0,
 X_Ariane_Teclado_1,
 X_Ariane_Teclado_10,
 X_Ariane_Teclado_11,
 X_Ariane_Teclado_12,
 X_Ariane_Teclado_2,
 X_Ariane_Teclado_3,
 X_Ariane_Teclado_4,
 X_Ariane_Tecl

In [135]:
# Cria o DataFrame result com os índices de V e as colunas de df
result = pd.DataFrame(0, index=df.index, columns=P.index)

# Iterando sobre as colunas
for j in df.columns:
    for i in df.index:
        for k in result.columns:
          if df.loc[i, j] == k:
                result.loc[i, k] = 1


In [137]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Peso dos Dias','Disponibilidade', 'Dias por Mês', 'Vozes'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
D = pd.Series(sheets_dict['Peso dos Dias']['Peso'])
Disp = sheets_dict['Disponibilidade']
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

In [146]:
from pulp import *

days = 2

# Definir o problema
prob_v = LpProblem("Alocacao_de_Cantores", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (V.index, V.columns, D.index), cat='Binary')

# Função objetivo
prob_v += lpSum([X[i][j][k] * V.loc[i, j] * L[j] * D[k] for i in V.index for j in V.columns for k in D.index])

# Restrições
for j in V.columns:
    for k in D.index:
        prob_v += lpSum([X[i][j][k] for i in V.index]) <= 1

for i in V.index:
    for k in D.index:
        prob_v += lpSum([X[i][j][k] for j in V.columns]) <= 1

for i in V.index:
    for k in range(len(D.index) - 3):
        prob_v += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]] for j in V.columns]) <= DPM['Quantidade'][i]

for i in V.index:
    for j in V.columns:
        for k in D.index:
            if V.loc[i, j] == 0:
                prob_v += X[i][j][k] == 0

# Resolver o problema
prob_v.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hmbarros/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/290e46bc3004452bbc85b8705442ee82-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/290e46bc3004452bbc85b8705442ee82-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 978 COLUMNS
At line 7819 RHS
At line 8793 BOUNDS
At line 9730 ENDATA
Problem MODEL has 973 rows, 936 columns and 4539 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1365 - 0.00 seconds
Cgl0002I 507 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 62 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 59 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 55 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 49 strengthened rows, 0 substitutions
Cgl0

1

In [149]:
# Imprimir o status da solução
print("Status:", LpStatus[prob_v.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob_v.objective))

# Dicionário para armazenar os resultados
resultados_v = {}

# Iterar sobre cada variável de decisão
for v in prob_v.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o voz e o dia
        _, pessoa, voz, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados_v:
            resultados_v[dia] = {}

        # Adicione a pessoa e o voz ao dia correspondente
        resultados_v[dia][voz] = pessoa

df_v = pd.DataFrame(columns=V.columns, index=sheets_dict['Peso dos Dias']['Dias']).fillna('')

print(prob_v.variables())
resultados_v

# Preencher o DataFrame com os resultados
# for dia in resultados:
#     for voz in resultados[dia]:
#         df_v.loc[sheets_dict['Peso dos Dias']['Dias'][int(dia)], voz] = resultados[dia][voz]


# if not pd.api.types.is_datetime64_any_dtype(df_v.index):
#     df_v.index = pd.to_datetime(df_v.index, origin='1899-12-30', unit='D')

# # Formatar o índice para o formato brasileiro
# df_v.index = df_v.index.strftime('%d/%m/%Y')

# df_v


Status: Optimal
Valor da função objetivo = 1365.0
[X_Ariane_Back_1_0, X_Ariane_Back_1_1, X_Ariane_Back_1_10, X_Ariane_Back_1_11, X_Ariane_Back_1_12, X_Ariane_Back_1_2, X_Ariane_Back_1_3, X_Ariane_Back_1_4, X_Ariane_Back_1_5, X_Ariane_Back_1_6, X_Ariane_Back_1_7, X_Ariane_Back_1_8, X_Ariane_Back_1_9, X_Ariane_Back_2_0, X_Ariane_Back_2_1, X_Ariane_Back_2_10, X_Ariane_Back_2_11, X_Ariane_Back_2_12, X_Ariane_Back_2_2, X_Ariane_Back_2_3, X_Ariane_Back_2_4, X_Ariane_Back_2_5, X_Ariane_Back_2_6, X_Ariane_Back_2_7, X_Ariane_Back_2_8, X_Ariane_Back_2_9, X_Ariane_Back_3_0, X_Ariane_Back_3_1, X_Ariane_Back_3_10, X_Ariane_Back_3_11, X_Ariane_Back_3_12, X_Ariane_Back_3_2, X_Ariane_Back_3_3, X_Ariane_Back_3_4, X_Ariane_Back_3_5, X_Ariane_Back_3_6, X_Ariane_Back_3_7, X_Ariane_Back_3_8, X_Ariane_Back_3_9, X_Ariane_Ministração_0, X_Ariane_Ministração_1, X_Ariane_Ministração_10, X_Ariane_Ministração_11, X_Ariane_Ministração_12, X_Ariane_Ministração_2, X_Ariane_Ministração_3, X_Ariane_Ministração_4, X_Ar

{'1_1': {'Back': 'Ariane'},
 '1_10': {'Back': 'Ariane'},
 '2_0': {'Back': 'Ariane'},
 '2_4': {'Back': 'Ariane'},
 '2_7': {'Back': 'Ariane'},
 'Back_1_9': {'Voz': 'Dani'},
 'Back_3_1': {'Voz': 'Dani'},
 'Back_3_3': {'Voz': 'Dani'},
 'Back_3_4': {'Voz': 'Dani'},
 'Back_3_7': {'Voz': 'Dani'},
 'Ministração_10': {'Voz': 'Dani'},
 'Ministração_6': {'Voz': 'Dani'},
 '1_5': {'Back': 'Everlyn'},
 '2_11': {'Back': 'Everlyn'},
 '3_10': {'Back': 'Everlyn'},
 '3_12': {'Back': 'Everlyn'},
 '3_2': {'Back': 'Everlyn'},
 '3_8': {'Back': 'Everlyn'},
 '1': {'Ministração': 'Everlyn'},
 '4': {'Ministração': 'Everlyn'},
 '7': {'Ministração': 'Everlyn'},
 '1_6': {'Back': 'Henrique'},
 '2_3': {'Back': 'Henrique'},
 '2_9': {'Back': 'Henrique'},
 '11': {'Ministração': 'Henrique'},
 '2': {'Ministração': 'Henrique'},
 '5': {'Ministração': 'Henrique'},
 '8': {'Ministração': 'Henrique'},
 '2_1': {'Back': 'Júlia'},
 '2_12': {'Back': 'Júlia'},
 '1_8': {'Back': 'Marcela'},
 '2_2': {'Back': 'Marcela'},
 '3_11': {'Back